In [1]:
import numpy as np
import pandas as pd 
import yfinance as yf 
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from datetime import date

In [2]:
import talib as ta 

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [97]:
end=date.today()
start=end-pd.Timedelta(days=50)
data=yf.download('^NSEI',start=start,end=end,interval="15m")

[*********************100%***********************]  1 of 1 completed


In [98]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-10-11 09:15:00+05:30,17867.550781,17912.050781,17843.900391,17896.300781,17896.300781,0
2021-10-11 09:30:00+05:30,17896.500000,17951.500000,17881.550781,17946.250000,17946.250000,0
2021-10-11 09:45:00+05:30,17946.550781,17975.300781,17928.000000,17970.199219,17970.199219,0
2021-10-11 10:00:00+05:30,17970.150391,17980.150391,17961.800781,17979.000000,17979.000000,0
2021-10-11 10:15:00+05:30,17979.250000,17986.000000,17962.250000,17968.949219,17968.949219,0
2021-10-11 10:30:00+05:30,17969.699219,17997.849609,17967.199219,17997.849609,17997.849609,0
2021-10-11 10:45:00+05:30,17997.400391,18000.050781,17975.500000,17986.599609,17986.599609,0
2021-10-11 11:00:00+05:30,17986.750000,18019.550781,17985.099609,18016.099609,18016.099609,0
2021-10-11 11:15:00+05:30,18016.000000,18019.400391,18005.449219,18015.900391,18015.900391,0


In [99]:
#converting to seperate date and time
data['Date'] = pd.to_datetime(data.index).date
data['Time'] = pd.to_datetime(data.index).time
data['Date']=data['Date'].astype('str')
data['Time']=data['Time'].astype('str')

In [100]:
df=data[['Date','Time','Open','High','Low','Close','Adj Close','Volume']]

In [101]:
df.head()

,Date,Time,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,,,
2021-10-11 09:15:00+05:30,2021-10-11,09:15:00,17867.550781,17912.050781,17843.900391,17896.300781,17896.300781,0
2021-10-11 09:30:00+05:30,2021-10-11,09:30:00,17896.500000,17951.500000,17881.550781,17946.250000,17946.250000,0
2021-10-11 09:45:00+05:30,2021-10-11,09:45:00,17946.550781,17975.300781,17928.000000,17970.199219,17970.199219,0
2021-10-11 10:00:00+05:30,2021-10-11,10:00:00,17970.150391,17980.150391,17961.800781,17979.000000,17979.000000,0
2021-10-11 10:15:00+05:30,2021-10-11,10:15:00,17979.250000,17986.000000,17962.250000,17968.949219,17968.949219,0


In [102]:
df['Value1']=ta.CCI(df.High, df.Low, df.Close,timeperiod=16) #timeperiod

In [103]:
df['Value2']=ta.CCI(df.High, df.Low, df.Close,timeperiod=44) #timeperiod

In [104]:
df['Value3']=df['Close'].ewm(span=5,adjust=False).mean() #span needs to be optimized

In [105]:
df['RSI']=ta.RSI(df.Close,timeperiod=14) #time period

In [106]:
df['Value5']=df['RSI'].rolling(12).max().shift(1) #12 needs optimization
df['Value6']=df['RSI'].rolling(12).min().shift(1)

In [107]:
df.dropna(axis=0,inplace=True)

df['BuyCondition']= np.where((df.Value1>0) & (df.Value2>0)& (df.Close>df.Value3),1,0)


df['SellCondition']=np.where((df.Value1<0) & (df.Value2<0) & (df.Close<df.Value3),-1,0)

df['BuyExit']=np.where((df.Value6>=85) & (df.RSI<df.Value6),'ExitBuy',0)

df['SellExit']=np.where((((df.RSI>df.Value5)&(df.Value5<30)) | (df.RSI<=20)),'SellExit',0)

class bactest(FinancialData):
    def columns(self,):
        self.value1=value1
        self.value2=value2
        self.value3=value3
        self.RSI=RSI
        self.value5=value5
        self.value6=value6
        
              
        df['Value1']=ta.CCI(df.High, df.Low, df.Close,timeperiod=value1) #timeperiod
        df['Value2']=ta.CCI(df.High, df.Low, df.Close,timeperiod=value2) #timeperiod
        df['Value3']=df['Close'].ewm(span=value3,adjust=False).mean() #span needs to be optimized
        df['RSI']=ta.RSI(df.Close,timeperiod=RSI) #time period
        df['Value5']=df['RSI'].rolling(value5).max().shift(1) #12 needs optimization
        df['Value6']=df['RSI'].rolling(value6).min().shift(1)
        df.dropna(axis=0,inplace=True)
    def condition(self):
        df['BuyCondition']= np.where((df.Value1>0) & (df.Value2>0)& (df.Close>df.Value3),1,0)
        df['SellCondition']=np.where((df.Value1<0) & (df.Value2<0) & (df.Close<df.Value3),-1,0)
    
    def exit(self):
        df['BuyExit']=np.where((df.Value6>=85) & (df.RSI<df.Value6),'ExitBuy',0)
        df['SellExit']=np.where((((df.RSI>df.Value5)&(df.Value5<30)) | (df.RSI<=20)),'SellExit',0)

In [128]:
df['signal'] =0
df['position'] = 0
df['trade_price'] = 0 #ur condition (OPEN)
df['trade_ret'] = 0 #ur logic of calcualting return
for i in range(0, len(df)):
    if df.iloc[i].Time >='09:15:00' and df.iloc[i].Time <='14:45:00':
        if df.iloc[i-1].position != 0:
            df.position.iloc[i] = df.iloc[i-1].position
            df.trade_price.iloc[i] = df.iloc[i-1].trade_price ##urs
            df.signal.iloc[i]= df.signal.iloc[i-1]
        # Entry long
        if df.position.iloc[i]==0 and (df.iloc[i].Value1>0) and (df.iloc[i].Value2>0) and (df.iloc[i].Close>df.iloc[i].Value3):
            df.position.iloc[i] = 1
            df.trade_price.iloc[i] = df.iloc[i].Close ##urs
            df.signal.iloc[i] = 'Buy'
        # Check take profit and stop loss for long position  
        elif df.iloc[i].position==1 and (df.iloc[i].Value6>=85) and (df.RSI<df.iloc[i].Value6):
            df.position.iloc[i] = 0
            #df.trade_ret.iloc[i] = df.iloc[i].Close / df.iloc[i].trade_price - 1  #urs
            df.trade_ret.iloc[i] = df.iloc[i].Close -df.iloc[i].trade_price 
            df.trade_price.iloc[i] = df.iloc[i].Close #urs
            df.signal.iloc[i] = 'Buy Exit'     
        # Entry short
        elif df.position.iloc[i]==0 and (df.iloc[i].Value1<0) & (df.iloc[i].Value2<0) & (df.iloc[i].Close<df.iloc[i].Value3):
            df.position.iloc[i] = -1
            df.trade_price.iloc[i] = df.iloc[i].Close ##urs
            df.signal.iloc[i] = 'Sell'
        # Check take profit and stop loss for short position  
        elif df.iloc[i].position==-1 and (df.iloc[i].RSI>df.iloc[i].Value5)and(df.iloc[i].Value5<30) or (df.iloc[i].RSI<=20):
                df.position.iloc[i] = 0
                #df.trade_ret.iloc[i] = df.iloc[i].trade_price / df.iloc[i].Close - 1  ##urs
                df.trade_ret.iloc[i] = df.iloc[i].trade_price -df.iloc[i].Close
                df.trade_price.iloc[i] = df.iloc[i].Close #urs
                df.signal.iloc[i] ='Sell Exit' #can even use string like exit trade
    if  df.iloc[i].Time=='15:00:00':
        if df.iloc[i-1].position == 1:
            df.position.iloc[i] = 0
            #df.trade_ret.iloc[i] = df.iloc[i].Close / df.iloc[i-1].trade_price - 1
            df.trade_ret.iloc[i] = df.iloc[i].Close - df.iloc[i-1].trade_price
            df.trade_price.iloc[i] = df.iloc[i].Close
            df.signal.iloc[i] = 'Buy sq'
        elif df.iloc[i-1].position == -1 and df.iloc[i].Time==df.Time[-2]:
            df.position.iloc[i] = 0
            df.trade_ret.iloc[i] = df.iloc[i-1].trade_price / df.iloc[i].Close - 1 
            df.trade_ret.iloc[i] = df.iloc[i-1].trade_price - df.iloc[i].Close  
            df.trade_price.iloc[i] = df.iloc[i].Close 
            df.signal.iloc[i] ='Sell sq'
            

In [129]:
df.tail(35)

,Date,Time,Open,High,Low,Close,Adj Close,Volume,Value1,Value2,Value3,RSI,Value5,Value6,signal,position,trade_price,trade_ret
Datetime,,,,,,,,,,,,,,,,,,
2021-11-25 13:00:00+05:30,2021-11-25,13:00:00,17467.750000,17523.000000,17463.300781,17516.150391,17516.150391,0,74.155656,-7.910799,17496.073274,55.360100,58.117738,37.526968,Sell,-1,17383.300781,0.000000
2021-11-25 13:15:00+05:30,2021-11-25,13:15:00,17516.949219,17532.550781,17492.650391,17496.699219,17496.699219,0,75.587505,0.827238,17496.281922,52.226131,58.117738,37.526968,Sell,-1,17383.300781,0.000000
2021-11-25 13:30:00+05:30,2021-11-25,13:30:00,17498.000000,17514.800781,17483.449219,17504.150391,17504.150391,0,59.139306,-7.241452,17498.904745,53.316380,58.117738,40.690410,Sell,-1,17383.300781,0.000000
2021-11-25 13:45:00+05:30,2021-11-25,13:45:00,17503.949219,17516.750000,17491.900391,17496.349609,17496.349609,0,56.069251,-6.414159,17498.053033,51.978980,58.117738,44.952219,Sell,-1,17383.300781,0.000000
2021-11-25 14:00:00+05:30,2021-11-25,14:00:00,17496.599609,17523.050781,17489.150391,17508.849609,17508.849609,0,63.324984,1.104627,17501.651892,53.971419,58.117738,48.174391,Sell,-1,17383.300781,0.000000
2021-11-25 14:15:00+05:30,2021-11-25,14:15:00,17508.699219,17547.949219,17506.550781,17532.199219,17532.199219,0,114.900773,30.381276,17511.834334,57.517268,58.117738,48.174391,Sell,-1,17383.300781,0.000000
2021-11-25 14:30:00+05:30,2021-11-25,14:30:00,17532.349609,17545.849609,17518.750000,17542.750000,17542.750000,0,132.818538,39.883092,17522.139556,59.052281,58.117738,48.174391,Sell,-1,17383.300781,0.000000
2021-11-25 14:45:00+05:30,2021-11-25,14:45:00,17543.400391,17563.849609,17539.099609,17555.099609,17555.099609,0,185.784440,60.651654,17533.126241,60.836053,59.052281,48.174391,Sell,-1,17383.300781,0.000000
2021-11-25 15:00:00+05:30,2021-11-25,15:00:00,17558.349609,17561.150391,17509.599609,17530.349609,17530.349609,0,100.486017,37.047856,17532.200697,55.607850,60.836053,48.174391,Sell sq,0,17530.349609,-147.048828


In [130]:
df['trade_ret'].sum()

51.509765625